# Cleaning the dataset 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re
import string
import time

pd.set_option('display.max_columns', 50)

### Read in Raw Dataset

In [2]:
# reading in fixed metadata

# set data types
cols = ['PROJECT_ID', 'ABSTRACT', 'FY.x', 'PROJECT_TERMS', 'PROJECT_TITLE', 'DEPARTMENT', 'AGENCY', 'IC_CENTER', 
        'PROJECT_NUMBER', 'PROJECT_START_DATE', 'PROJECT_END_DATE', 'CONTACT_PI_PROJECT_LEADER', 'OTHER_PIS', 
        'CONGRESSIONAL_DISTRICT', 'DUNS_NUMBER', 'ORGANIZATION_NAME', 'ORGANIZATION_CITY', 'ORGANIZATION_STATE', 
        'ORGANIZATION_ZIP', 'ORGANIZATION_COUNTRY', 'BUDGET_START_DATE', 'BUDGET_END_DATE', 'CFDA_CODE', 'FY.y', 
        'FY_TOTAL_COST', 'FY_TOTAL_COST_SUB_PROJECTS']
dtypes = {col: 'str' for col in cols}
dtypes["FY_TOTAL_COST"] = 'float'
dtypes["FY_TOTAL_COST_SUB_PROJECTS"] = 'float'

df = pd.read_csv('../../../data/prd/Federal_RePORTER/FR_raw_2021FEB24.csv', dtype = dtypes, engine='python')
print(df.shape)

(1156137, 26)


In [3]:
df.head()

,PROJECT_ID,ABSTRACT,FY.x,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS
0,89996,"This is a project to explore Game-based, Metap...",2008,Achievement; analog; base; Cognitive Science;...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN
1,89997,Institution: Franklin Institute Science Museum...,2008,Active Learning; Child; Computer software; de...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN
2,89998,Through programs (including small group conver...,2008,Address; Age; Birth; Brain; Caregivers; Child...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN
3,89999,In partnership with the American Chemical Soci...,2008,Advanced Development; American; Chemicals; Ch...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN
4,90000,Amphibian populations around the world are exp...,2008,Amphibia; Central America; Communicable Disea...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,9/30/2011,"ZAMUDIO, KELLY R",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.074,2008,370996.0,NaN


In [4]:
# FY.x is the reliable fiscal year information so we rename this column to FY

df = df.rename(columns={'FY.x': 'FY'})

In [5]:
df.dtypes

PROJECT_ID                     object
ABSTRACT                       object
FY                             object
PROJECT_TERMS                  object
PROJECT_TITLE                  object
DEPARTMENT                     object
AGENCY                         object
IC_CENTER                      object
PROJECT_NUMBER                 object
PROJECT_START_DATE             object
PROJECT_END_DATE               object
CONTACT_PI_PROJECT_LEADER      object
OTHER_PIS                      object
CONGRESSIONAL_DISTRICT         object
DUNS_NUMBER                    object
ORGANIZATION_NAME              object
ORGANIZATION_CITY              object
ORGANIZATION_STATE             object
ORGANIZATION_ZIP               object
ORGANIZATION_COUNTRY           object
BUDGET_START_DATE              object
BUDGET_END_DATE                object
CFDA_CODE                      object
FY.y                           object
FY_TOTAL_COST                 float64
FY_TOTAL_COST_SUB_PROJECTS    float64
dtype: objec

In [6]:
df.isna().sum()

PROJECT_ID                          0
ABSTRACT                        42380
FY                              35035
PROJECT_TERMS                   35278
PROJECT_TITLE                       0
DEPARTMENT                          0
AGENCY                              0
IC_CENTER                      194697
PROJECT_NUMBER                      0
PROJECT_START_DATE             154926
PROJECT_END_DATE               146332
CONTACT_PI_PROJECT_LEADER          45
OTHER_PIS                     1012481
CONGRESSIONAL_DISTRICT          74481
DUNS_NUMBER                     14569
ORGANIZATION_NAME                1660
ORGANIZATION_CITY                5926
ORGANIZATION_STATE              17001
ORGANIZATION_ZIP                48989
ORGANIZATION_COUNTRY             5698
BUDGET_START_DATE              256052
BUDGET_END_DATE                256116
CFDA_CODE                      201351
FY.y                                0
FY_TOTAL_COST                  212739
FY_TOTAL_COST_SUB_PROJECTS     982306
dtype: int64

### Remove null abstracts

In [7]:
#remove rows with NULL abstracts

l1 = len(df)
df = df[~df.ABSTRACT.isnull()]
l2 = len(df)

print(l1-l2, "null ABSTRACTs removed")

42380 null ABSTRACTs removed


### Fill in missing start dates

**Since we are tracking emerging abstract trends**, we will fill in missing information for:  
- PROJECT_START_DATE
- PROJECT_END_DATE: not filling in project end dates -- we didn't use it and it doesn't make sense to fill in FY for the end date nor to sort by it

In [8]:
print(sum(df["PROJECT_START_DATE"].isna()))
print(100*sum(df["PROJECT_START_DATE"].isna())/len(df))

150344
13.498815271194704


In [9]:
# Fill missing values in Project Start/End Date with Budget Start/End Date
df['PROJECT_START_DATE'] = df['PROJECT_START_DATE'].fillna(df['BUDGET_START_DATE'])

#df['PROJECT_END_DATE'] = df['PROJECT_END_DATE'].fillna(df['BUDGET_END_DATE'])

In [10]:
print(sum(df["PROJECT_START_DATE"].isna()))
print(100*sum(df["PROJECT_START_DATE"].isna())/len(df))

41813
3.7542300519772267


In [11]:
100*(150344-41813)/len(df)

9.744585219217477

In [12]:
#If START date is still missing, fill with FY
df['PROJECT_START_DATE'] = df['PROJECT_START_DATE'].fillna(df['FY'])
#df['PROJECT_END_DATE'] = df['PROJECT_END_DATE'].fillna(df['FY'])

In [13]:
print(sum(df["PROJECT_START_DATE"].isna()))
print(100*sum(df["PROJECT_START_DATE"].isna())/len(df))

0
0.0


In [14]:
100*(41813-0)/len(df)

3.7542300519772267

### Aggregate Counts

In [15]:
# strip white space from character columns

for col in df.columns[0:24]:
    print(col)
    df[col] = df[col].str.strip()

PROJECT_ID
ABSTRACT
FY
PROJECT_TERMS
PROJECT_TITLE
DEPARTMENT
AGENCY
IC_CENTER
PROJECT_NUMBER
PROJECT_START_DATE
PROJECT_END_DATE
CONTACT_PI_PROJECT_LEADER
OTHER_PIS
CONGRESSIONAL_DISTRICT
DUNS_NUMBER
ORGANIZATION_NAME
ORGANIZATION_CITY
ORGANIZATION_STATE
ORGANIZATION_ZIP
ORGANIZATION_COUNTRY
BUDGET_START_DATE
BUDGET_END_DATE
CFDA_CODE
FY.y


**Aggregate counts for unique ORGANIZATION_NAMEs in rows with duplicated Abstract/Title/Project_Start_Date**

In [16]:
# group df by abstract/title/start_date (exact matches)
all_grp = df.groupby(['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'])

# for each unique ABSTRACT/TITLE/START_DATE in df, count unique Organizations
unique_all = all_grp.agg({'ORGANIZATION_NAME' : 'nunique'}) 

#rename column as "count" to be different than original column
unique_all = unique_all.rename(columns={'ORGANIZATION_NAME': 'ORG_COUNT'})

# merge df with "unique_all" to bring in the unique Organization counts for each "duplicate" group
merged1 = df.merge(unique_all, how='left', on=['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'])

**Aggregate counts for unique PIs in rows with duplicated Abstract/Title/Project_Start_Date**

In [17]:
# for each unique ABSTRACT/TITLE/START_DATE in whole df, count unique PIs
unique_pi = all_grp.agg({'CONTACT_PI_PROJECT_LEADER' : 'nunique'}) 

#rename column as "count" to be different than original column
unique_pi = unique_pi.rename(columns={'CONTACT_PI_PROJECT_LEADER': 'PI_COUNT'})

# merge data frame with "unique_all" to bring in the unique PI counts for each "duplicate" group
merged = merged1.merge(unique_pi, how='left', on=['ABSTRACT', 'PROJECT_TITLE', 'PROJECT_START_DATE'])

In [18]:
# Sort merged data so that duplicated rows occur in order of earliest to latest END date
#merged = merged2.sort_values(['PROJECT_END_DATE'])

In [18]:
merged.head()

,PROJECT_ID,ABSTRACT,FY,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT
0,89996,"This is a project to explore Game-based, Metap...",2008,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1
1,89997,Institution: Franklin Institute Science Museum...,2008,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1
2,89998,Through programs (including small group conver...,2008,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1
3,89999,In partnership with the American Chemical Soci...,2008,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1
4,90000,Amphibian populations around the world are exp...,2008,Amphibia; Central America; Communicable Diseas...,COLLABORATIVE RESEARCH: EVOLUTION OF AMPHIBIAN...,NSF,NSF,NaN,0815315,10/1/2008,9/30/2011,"ZAMUDIO, KELLY R",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.074,2008,370996.0,NaN,2,2


### Deduplication

In [19]:
print(sum(merged['ABSTRACT'].isnull()))
print(sum(merged['PROJECT_TITLE'].isnull()))
print(sum(merged['PROJECT_START_DATE'].isnull()))

0
0
0


In [20]:
# save NON-duplicated rows and the LAST occurrance of duplicated rows
dedup = merged[~merged.duplicated(subset=['ABSTRACT',  'PROJECT_TITLE', 'PROJECT_START_DATE'], keep='last')]

In [21]:
print(f"Original: {len(df)}")
print(f"Deduplicated: {len(dedup)}")
print(f"Number of Duplicates: {len(df) - len(dedup)}")

Original: 1113757
Deduplicated: 699652
Number of Duplicates: 414105


In [22]:
#save final (deduplicated) dataframe as "df" to fit downstream code
df = dedup

### Dataset setup

##### Helper function used throughout

In [23]:
# drops strings with length 0 - indices printed out, find with df.loc[ix]

def drop_empties(df, col):
    
    l1 = len(df)
    ix = df[df[col].apply(len)==0].index
    print(ix)
    df.drop(ix,axis=0,inplace=True)
    l2 = len(df)
    
    print(f"dropped {l1-l2}")
    
    return df

In [24]:
# strip leading and trailing whitespace, save in a working abstract column that will be updated as text is cleaned
# strip() already ran above

# Note: we cannot lower case abstracts up front - capitalization is needed to find POS in preprocessing
 
#df = df.assign(working_abstract = [abstract.strip() for abstract in df["ABSTRACT"]])
df["working_abstract"] = df["ABSTRACT"]
df = drop_empties(df, "working_abstract")

wa = 'working_abstract'

<ipython-input-24-61475bfdf4cd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["working_abstract"] = df["ABSTRACT"]


Int64Index([], dtype='int64')
dropped 0


/apps/software/standard/compiler/gcc/9.2.0/jupyter_conda/2020.11-py3.8/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [25]:
df = df.assign(Start_Char = df['working_abstract'].apply(lambda x: x[0]))

In [26]:
def remove_short_abstracts(df, limit):

    # Remove abstracts with length < limit. 150 seems like a good cutoff, but it does lose some useful information.
    
    # what do we want to do for the cutoff?  -- SOLUTION UNTIL FURTHER EXPLORATION -> keep the same
    
    df['nchar']=df['working_abstract'].apply(len)
    l1 = len(df)
    df=df.loc[df['nchar']>=limit]
    l2 = len(df)
    
    print(l1-l2, "short abstracts removed")
    
    return df

df = remove_short_abstracts(df,limit=150)

3520 short abstracts removed


In [27]:
len(df)

696132

### Cleaning Strategy:
1. Remove abstracts with all non-alphanumeric characters.
2. Remove non-alphanumeric characters from the start and end of abstracts
3. Remove other non-readable abstracts. (REMOVAL ABSTRACTS FOUND BY INSPECTION)
4. Remove "junk" starting strings and ending strings
5. Remove "junk" strings in the middle 
6. Remove title and organization name from abstracts


##### Helper functions for cleaning

In [28]:
def strip_nonalnum(word):
    
    # function strips non-alphanumeric characters from the beginning and end of a string
    # adapted from: https://stackoverflow.com/questions/22650506/how-to-rermove-non-alphanumeric-characters-at-the-beginning-or-end-of-a-string
        
    if not word:
        return word  # nothing to strip
    if (len(word) == 1) and (not word[0].isalnum()):  
        return ""
    for start, c in enumerate(word):
        if c.isalnum():
            break
    for end, c in enumerate(word[::-1]):
        if c.isalnum():
            break
                  
    return word[start:len(word) - end]


In [29]:
def remove_phrase(x, phrase,loc='Start'):
    
    # returns x with phrase removed. location can be "Start" of string, "End" of string, or 
    # "Anywhere_All"--anywhere will remove all instances and Anywhere_First will remove the first instance
    # CASE info - this function assumes phrase is lower case, but that x is not.
    
    assert loc in ['Start','End']
    
    if loc=='End':
        if x.lower().endswith(phrase):
            return x[:-1*len(phrase)].strip()
        else:
            return x
    elif loc=='Start':
        if x.lower().startswith(phrase):
            return x[len(phrase):].strip()
        else:
            return x
    else:
        return 'Error'
    

In [30]:
# strip non-alphanum characters from the beginning and end of each abstract

temp = [strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = df.assign(working_abstract = temp)

df = drop_empties(df, "working_abstract")

Int64Index([96789, 668610, 756384], dtype='int64')
dropped 3


In [31]:
# remove abstracts that are not readable: THIS NEEDS TO BE UPDATED BY HAND FOR EVERY NEW DATASET
# For example: index = 490684: ¢ £/¥ ƒ § ¤ ƒ “ ƒ « ...

df = df.assign(Start_Char = df['working_abstract'].apply(lambda x: x[0]))
ix = df[df['Start_Char'] == 'ƒ'].index
df.drop(index = ix, inplace = True)

In [32]:
ix

Int64Index([993838], dtype='int64')

In [33]:
# "junk" phrases at start to remove

start_phrases=['abstract', 'summary', 'proposal', 'description', 'narrative', 
               'technical abstract',
               'non technical abstract', 
               'non- technical abstract',
               'non-technical abstract',                      
               'nontechnical abstract',
               'technical summary', 
               'nontechnical summary',
               'non-technical summary',
               'non-technical description',
               'description (provided by the applicant)',
               'description (provided by investigator)',  
               'description (provided by applicant)',
               'project summary/abstract',
               'proposal abstract',
               'research abstract',
               'project summary',
               'research summary',
               'project description'
               'see instructions):',
               'for center application (provided by the investigator):',
               'objective(s)',      
               'exceed the space provided',
               'provided by applicant',
               'provided by candidate']


In [34]:
#Remove found start phrases

for phrase in start_phrases:
    temp = df[wa].apply(remove_phrase,args=[phrase,'Start'])
    df = df.assign(working_abstract = temp) 

# strip non-alphanum characters from the beginning and end of each abstract

temp = [strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = df.assign(working_abstract = temp)

df = drop_empties(df, "working_abstract")

Int64Index([], dtype='int64')
dropped 0


In [35]:
#Repeated start phrase removal in case the order of project summary/abstract varies

for phrase in start_phrases:
    temp = df[wa].apply(remove_phrase,args=[phrase,'Start'])
    df = df.assign(working_abstract = temp)  
    
df = drop_empties(df, "working_abstract")

Int64Index([], dtype='int64')
dropped 0


In [36]:
#starting_exact_phrases to remove

#'This subproject represents an estimate of the percentage of the CTSA funding that isbeing utilized for a broad area of research (AIDS research, pediatric research, orclinical trials).  The Total Cost listed is only an estimate of the amount of CTSAinfrastructure going towards this area of research, not direct funding provided bythe NCRR grant to the subproject or subproject staff.'
#'This subproject is one of many research subprojects utilizing theresources provided by a Center grant funded by NIH/NCRR. The subproject andinvestigator (PI) may have received primary funding from another NIH source,and thus could be represented in other CRISP entries. The institution listed isfor the Center, which is not necessarily the institution for the investigator.'

temp = df[wa].apply(lambda x: x.replace('This subproject represents an estimate of the percentage of the CTSA funding that isbeing utilized for a broad area of research (AIDS research, pediatric research, orclinical trials).  The Total Cost listed is only an estimate of the amount of CTSAinfrastructure going towards this area of research, not direct funding provided bythe NCRR grant to the subproject or subproject staff.',
                                       ''))
df = df.assign(working_abstract = temp) 

expression=re.compile('This subproject is one of many research subprojects.*not necessarily the institution for the investigator.')
temp = df[wa].apply(lambda x: re.sub(expression,'',x))
df = df.assign(working_abstract = temp)

expression=re.compile('This subproject is one of many research subprojects.*to the subproject or subproject staff.')
temp = df[wa].apply(lambda x: re.sub(expression,'',x))
df = df.assign(working_abstract = temp)

In [37]:
# strip non-alphanum characters from the beginning and end of each abstract

temp = [strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = df.assign(working_abstract = temp)

df = drop_empties(df, "working_abstract")

# update Start_Char column in df
df = df.assign(Start_Char = df['working_abstract'].apply(lambda x: x[0]))

Int64Index([], dtype='int64')
dropped 0


In [38]:
end_phrases = ['(end of abstract',
               'end of abstract', 
               '(abstract end',  
               '(end of abstract',
               '(end 0f abstract',
               '(end of absract',
               '(abstract below',
               '(end of reviewers\' comment',
               '(end abstract',
               'performance site ========================================section end',
               'key personnel ========================================section end',
               '[summary truncated at 7800 characters', 
               'this award reflects nsf\'s statutory mission and has been deemed worthy of support through evaluation using the foundation\'s intellectual merit and broader impacts review criteria',
               'project description page 6', 'page 1 of 1', 'project summary/abstract page 6',
               'project description page 7', 'project summary/abstract page 7', 'pag 1 o 1', 
               'page 2 number pages consecutively at the bottom throughout form page 2']


In [39]:
# end phrase removal

for phrase in end_phrases:
    temp = df[wa].apply(remove_phrase,args=[phrase,'End'])
    df = df.assign(working_abstract = temp) 

# strip non-alphanum characters from the beginning and end of each abstract

temp = [strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = df.assign(working_abstract = temp)

df = drop_empties(df, "working_abstract")
    
# update Last Char column in df -- this actually sets last char to start char
df = df.assign(LAST_CHAR = df['working_abstract'].apply(lambda x: x[0]))

Int64Index([], dtype='int64')
dropped 0


In [40]:
# "junk" removal within text body - not necessarily at the start or end

# 'Enter the text here that' ending with 'lines of text.'
expression=re.compile('Enter the text here that.*lines of text')
temp=df[wa].apply(lambda x: re.sub(expression,'',x))
df = df.assign(working_abstract = temp)

expression=re.compile('PHS .*?Continuation Format Page')
temp=df[wa].apply(lambda x: re.sub(expression,'',x))
df = df.assign(working_abstract = temp)

expression=re.compile('OMB No .*?Continuation Format Page')
temp=df[wa].apply(lambda x: re.sub(expression,'',x))
df = df.assign(working_abstract = temp)

temp=df[wa].replace('Project Summary/Abstract','')
df = df.assign(working_abstract = temp)

In [41]:
"""If it starts with 'one page and must contain',
This is an NIH thing and there aren't that many of them, but come from 3 different cfda
it will start with "one page and must contain a summary of the proposed activity suitable for dissemination to 
thepublic. It should be a self-contained description of the project and should contain a statement of objectives 
and methods to be employed. It should be informative to other persons working in the same or related fields and 
insofar as possible understandable to a technically liter-ate lay reader. This Abstract must not include any 
proprietary/confidential information.* Please click the add attachment button to complete this entry." plus some 
attachments, which includes tracking number, twice: following the second trackign number, there is a grant number
followed by the actual content" 

At the end of these files, they all end in 'Project Narrative File'(last instance) followed by more attachments, 
all of which can be discarded
"""

expression1=re.compile('one page and must.*?Tracking Number.*?(Tracking Number)')
expression2=re.compile('Project Narrative File.*')

def fix_abstract(abstract):
    if abstract.startswith('one page and must contain'):
        abstract=re.sub(expression1,'',abstract)
        return re.sub(expression2,'',abstract)
    else:
        return abstract

temp=df[wa].apply(fix_abstract)
df = df.assign(working_abstract = temp)

In [42]:
# removal of phrase at end or beginning?

expression=re.compile('Project Summary/Abstract Page.*')

def remove_contact_pd(x):
    
    """removes clause at end that tends to occur: eg Project Summary/Abstract Page 222Contact PD/PI: Sampson, HughNarrative ("""
    
    if x.startswith('Contact PD/PI'):
        return re.sub(expression,'',x)
    else:
        return x
    
temp=df[wa].apply(remove_contact_pd) 
df = df.assign(working_abstract = temp)

In [43]:
# strip non-alphanum characters from the beginning and end of each abstract

temp = [strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = df.assign(working_abstract = temp)

df = drop_empties(df, "working_abstract")

Int64Index([], dtype='int64')
dropped 0


In [44]:
len(df)

696128

In [45]:
df.to_pickle("../../../data/prd/Tech-Report/FR_clean_21SEPT07.pkl")


### Scratch Work

Decided not to remove titles and org names - this removed 10,308 abstracts.  

If we remove titles, we lose ~10,000 of the 16,000 NASA abstracts. 
Only ~9500 abstracts contain titles and 679 abstracts contain org names.  Not a strong enough signal to appear in topic model on full dataset.


In [49]:
# count how many titles appear in abstracts - will not count titles that end in '\' - none of them do!

count = 0
for i, abstract in enumerate(df['working_abstract']):
    title = df.iloc[i]['PROJECT_TITLE']
    #c = title[-1]   
    if title in abstract:
        #print(i)
        count = count + 1
    
print(count)

9525


In [50]:
# count how many org names appear in abstracts

count = 0
for i, abstract in enumerate(df['working_abstract']):
    title = df.iloc[i]['ORGANIZATION_NAME']
    
    if type(title) == float and np.isnan(title):
        # do nothing
        count = count + 0
    else:
        #c = title[-1]
        if title in abstract:
            count = count + 1
    
print(count)

679


In [91]:
# breakdown of abstracts that would have been removed
# these counts are a little (~600) different than the counts in the two cells above.  Could be due to escape chars?

R['DEPARTMENT'].value_counts()

NASA    10129
HHS       179
Name: DEPARTMENT, dtype: int64

In [92]:
sum(R["PROJECT_TITLE"].apply(len))/len(R)

251.2593131548312

In [93]:
sum(df['PROJECT_TITLE'].apply(len))/len(df)

66.21136158175615

In [ ]:
# length of titles in the removed indices is much longer than a typical title
# suggests these may not be actual titles

In [183]:
# NASA example - common structure

i = 286738
print(df.iloc[i]['ABSTRACT'])
print('------')
print(df.iloc[i]['working_abstract'])
print('------')
print(df.iloc[i]['PROJECT_TITLE'])

WE PROPOSE TO STUDY SUPPORTING RADIO-FREQUENCY TECHNOLOGY FOR THE NEXT GENERATION OF BALLOON-BORNE OBSERVATORIES FOR ULTRA-HIGH ENERGY (UHE) NEUTRINOS AND OTHER COSMIC PARTICLES IN THE EXAVOLT (10^{18} EV) ENERGY RANGE. THESE NEW METHODOLOGIES OBSERVE NAN
------
WE PROPOSE TO STUDY SUPPORTING RADIO-FREQUENCY TECHNOLOGY FOR THE NEXT GENERATION OF BALLOON-BORNE OBSERVATORIES FOR ULTRA-HIGH ENERGY (UHE) NEUTRINOS AND OTHER COSMIC PARTICLES IN THE EXAVOLT (10^{18} EV) ENERGY RANGE. THESE NEW METHODOLOGIES OBSERVE NAN
------
WE PROPOSE TO STUDY SUPPORTING RADIO-FREQUENCY TECHNOLOGY FOR THE NEXT GENERATION OF BALLOON-BORNE OBSERVATORIES FOR ULTRA-HIGH ENERGY (UHE) NEUTRINOS AND OTHER COSMIC PARTICLES IN THE EXAVOLT (10^{18} EV) ENERGY RANGE. THESE NEW METHODOLOGIES OBSERVE NAN


In [125]:
# HHS example - common structure

i = 154
print(rtemp.iloc[i]['ABSTRACT'])
print('------')
print(rtemp.iloc[i]['working_abstract'])
print('------')
print(rtemp.iloc[i]['PROJECT_TITLE'])

This subproject is one of many research subprojects utilizing the resourcesprovided by a Center grant funded by NIH/NCRR. Primary support for the subprojectand the subproject's principal investigator may have been provided by other sources,including other NIH sources.  The Total Cost listed for the subproject likelyrepresents the estimated amount of Center infrastructure utilized by the subproject,not direct funding provided by the NCRR grant to the subproject or subproject staff.Training in the Use of Bruker and Varian Spectrometers and NMR
------
Training in the Use of Bruker and Varian Spectrometers and NMR
------
TRAINING IN THE USE OF BRUKER AND VARIAN SPECTROMETERS AND NMR


In [ ]:
# code to remove titles and organizations below

In [191]:
def remove_title_org(record):
    
    # This function removes project titles and organization names from abstracts
    # CORRECTED TO ADDRESS COMMENT BELOW - escape characters were the problem
    
    """ ignores case to remove multi-word phrases in a particular order, especially those likely to run into other words,
    e.g. Institution university of washingtonPI mary williams. This doesn't work when titles or insititutions have escape characters in them, which is a bummer
    see for example ENHANCING THE USE OF NASA EARTH SCIENCE RESULTS / DATA / AND TECHNOLOGY BY ENGAGING THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS COMMUNITIES OF
    PRACTICE IN TARGET AREAS OF INTEREST TO NASA THE FEDERATION OF EARTH SCIENCE INFORMATION PARTNERS (''FED"""
    
    title=record['PROJECT_TITLE']
    org_name = record['ORGANIZATION_NAME']
    
    # correct escape sequences for re.sub()
    
    if '\\' in title:
        title = title.replace('\\', '\\\\')
    if '(' in title:
        title = title.replace('(', '\(')
    if ')' in title:
        title = title.replace(')', '\)')
    if '+' in title:
        title = title.replace('+', '\+')
    if '[' in title:
        title = title.replace('[', '\[')
    if ']' in title:
        title = title.replace(']', '\]')
    if '?' in title:
        title = title.replace('?', '\?')
    if '*' in title:
        title = title.replace('*', '\*')
    if '^' in title:
        title = title.replace('^', '\^')
    if '}' in title:
        title = title.replace('}', '\}')
   
    new_abstract = re.sub(title,'',record[wa],flags=re.IGNORECASE)
    
    if (type(org_name) == float) and np.isnan(org_name):  # case where org_name = nan
        return new_abstract 
    else:        
        if '\\' in org_name:
            org_name = org_name.replace('\\', '\\\\')
        if '(' in org_name:
            org_name = org_name.replace('(', '\(')
        if ')' in org_name:
            org_name = org_name.replace(')', '\)')
            
        return re.sub(org_name, '', new_abstract, flags=re.IGNORECASE)        

In [86]:
# run function        
temp=df.apply(lambda x: remove_title_org(x),axis=1)
df = df.assign(working_abstract = temp)

In [88]:
# strip non-alphanum characters from the beginning and end of each abstract

temp = [strip_nonalnum(abstract) for abstract in df["working_abstract"]]
df = df.assign(working_abstract = temp)

df, ix = drop_empties(df, "working_abstract")

df = df.assign(Start_Char = df[wa].apply(lambda x:x[0]))
df = df.assign(LAST_CHAR = df[wa].apply(lambda x:x[-1]))
df = df.assign(nchar = df[wa].apply(lambda x: len(x)))

Int64Index([     51,      63,      72,      90,     111,     131,     135,
                150,     151,     156,
            ...
             934785,  934792,  934793,  934811,  934827,  934871,  934891,
             934897,  941388, 1076477],
           dtype='int64', length=10308)
dropped 10308
